In [105]:
import pandas as pd
from random import sample
import random
import numpy as np

random.seed(0)
para_train = pd.read_csv('./data/quora-train.csv', sep="\t")
sts_train = pd.read_csv('./data/sts-train.csv', sep="\t")
para_train = para_train.dropna()
para_train = para_train.head(1000) #!!!!
sts_train = sts_train.dropna()
para_dev = pd.read_csv('./data/quora-dev.csv', sep="\t")
sts_dev = pd.read_csv('./data/sts-dev.csv', sep="\t")
para_dev = para_dev.dropna()
para_dev = para_dev.head(10) #!!!!
sts_dev = sts_dev.dropna() 
para_test = pd.read_csv('./data/quora-test-student.csv', sep="\t")
sts_test = pd.read_csv('./data/sts-test-student.csv', sep="\t")
para_test = para_test.dropna()
sts_test = sts_test.dropna() 

In [4]:
from sentence_transformers import SentenceTransformer
# sentences = ["This is an example sentence", "Each sentence is converted"]

# model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
# embeddings = model.encode(sentences)
# print(embeddings)
# print(para_train.head())

/opt/conda/envs/cs229/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [117]:
from torch import nn
import torch
import torch.nn.functional as F
import torch.utils.data as data_utils
from torch.utils.data import DataLoader, Dataset


N_PARAPHRASE_CLASSES = 1
N_SIMILARITY_CLASSES = 5
DROPOUT_PROB = 0.5
INPUT_SIZE = 768

class NLP_Model(nn.Module):
    def __init__(self, model):
        super(NLP_Model, self).__init__()
        self.model = model
        self.dropout = nn.Dropout(DROPOUT_PROB)
        self.paraphrase_linear = nn.Linear(INPUT_SIZE, INPUT_SIZE // 2)
        self.paraphrase_linear_interact = nn.Linear(INPUT_SIZE, N_PARAPHRASE_CLASSES)
        self.similarity_linear = nn.Linear(INPUT_SIZE, INPUT_SIZE // 2)
        self.similarity_linear_interact = nn.Linear(INPUT_SIZE, N_SIMILARITY_CLASSES)
    
    def forward(self, sentences1, sentences2, task, device):
        '''
        Task 0 is para. Task 1 is similarity.
        '''
        sentences1 = torch.as_tensor(self.model.encode(sentences1.tolist()))
        sentences1 = sentences1.to(device)
        sentences2 = torch.as_tensor(self.model.encode(sentences2.tolist()))
        sentences2 = sentences2.to(device)
        if task == 0:
            sentences1 = self.dropout(sentences1)
            sentences1 = F.relu(self.paraphrase_linear(sentences1))
            sentences2 = self.dropout(sentences2)
            sentences2 = F.relu(self.paraphrase_linear(sentences2))
            combined = torch.concat((sentences1, sentences2), dim=-1)
            combined = self.dropout(combined)
            return F.sigmoid(self.paraphrase_linear_interact(combined))
        if task == 1:
            sentences1 = self.dropout(sentences1)
            sentences1 = F.relu(self.similarity_linear(sentences1))
            sentences2 = self.dropout(sentences2)
            sentences2 = F.relu(self.similarity_linear(sentences2))
            combined = torch.concat((sentences1, sentences2), dim=-1)
            combined = self.dropout(combined)
            return F.softmax(self.similarity_linear_interact(combined), dim=-1)


In [130]:
from torch.optim import AdamW

def save_model(model, optimizer, filepath):
    save_info = {
        'model': model.state_dict(),
        'optim': optimizer.state_dict(),
        'system_rng': random.getstate(),
        'numpy_rng': np.random.get_state(),
        'torch_rng': torch.random.get_rng_state(),
    }

    torch.save(save_info, f'{filepath}/model')
    model.model.save(f'{filepath}/transformer')
    print(f"saved the model to {filepath}")

def load_model(filepath, device):
    with torch.no_grad():
        save_info = torch.load(f'{filepath}/model')
        transformer_model = SentenceTransformer(f'{filepath}/transformer')
        transformer_model.to(device)
        
        model = NLP_Model(transformer_model)
        model.load_state_dict(save_info['model'])
        model.to(device)
        
        optimizer = AdamW(list(model.parameters()) + list(transformer_model.parameters()), lr=1e-4)
        optimizer.load_state_dict(save_info['optim'])
        
        random.setstate(save_info['system_rng'])
        np.random.set_state(save_info['numpy_rng'])
        torch.random.set_rng_state(save_info['torch_rng'])
    return model, optimizer

In [119]:
from math import ceil

def get_batches(dataset, batch_size=256):
    """
    Pass in dataset and batch size.
    Get generator which yields batches.
    """
    return enumerate(dataset[i*batch_size:(i+1)*batch_size] for i in range(ceil(dataset.shape[0] / batch_size)))

In [124]:
from torch.optim import AdamW
from tqdm import tqdm
NUM_EPOCHS = 10

def train_singletask_para_model(para_train, para_dev, filepath):
    '''
    use AdamW optimizer.
    binary cross-entropy loss.
    make sure to save model at end to a specific path.
    '''
    device = torch.device('cuda')
    
    transformer = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
    transformer.to(device)

    model = NLP_Model(transformer)
    model = model.to(device)

    optimizer = AdamW(list(model.parameters()) + list(transformer.parameters()), lr=1e-4) #~SGD with weight decay 0.01
    best_dev_acc = 0

    train_para_accuracy = eval_singletask_model(model, device, para_train, 0, 'train')
    dev_para_accuracy = eval_singletask_model(model, device, para_dev, 0, 'dev')
    print(f"epoch number: 0, para train accuracy: {train_para_accuracy}, para dev accuracy: {dev_para_accuracy}")

    for epoch in range(NUM_EPOCHS):
        model.train()
        transformer.train()

        for step, batch in tqdm(get_batches(para_train), desc='train'):
            b_sentences1, b_sentences2, b_labels = batch['sentence1'], batch['sentence2'], batch['is_duplicate']
            optimizer.zero_grad()
            logits = model.forward(b_sentences1, b_sentences2, 0, device).flatten()

            b_labels = torch.as_tensor(b_labels.values, dtype=torch.float32)
            b_labels = b_labels.to(device)

            loss = F.binary_cross_entropy(logits, b_labels, reduction='mean')
            loss.backward()
            optimizer.step()

        train_para_accuracy = eval_singletask_model(model, device, para_train, 0, 'train')
        dev_para_accuracy = eval_singletask_model(model, device, para_dev, 0, 'dev')
        print(f"epoch number: {epoch + 1}, para train accuracy: {train_para_accuracy}, para dev accuracy: {dev_para_accuracy}")

        if dev_para_accuracy > best_dev_acc:
            best_dev_acc = dev_para_accuracy
            print('New best model. Saving.')
            save_model(model, optimizer, filepath)


def train_singletask_sts_model():
    '''
    use AdamW optimizer.
    multi-class cross-entropy loss.
    make sure to save model at end to a specific path.
    '''
    device = torch.device('cuda')

    sts_train_dataloader = DataLoader(sts_train, shuffle=True, batch_size=16)
    sts_dev_dataloader = DataLoader(sts_dev, shuffle=True, batch_size=16)
    
    transformer = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
    transformer.to(device)
    #transformer.train()

    pass

https://huggingface.co/docs/transformers/training#train-in-native-pytorch

In [125]:
def eval_singletask_model(model, device, dataset, task, flag):
    '''
    given dataloader, 2 task-specific finetuned models, and device
    return the accuracy for para and for sts
    '''
    model.eval()
    model.model.eval()
    with torch.no_grad():
        truth = []
        predictions = []
        for step, batch in tqdm(get_batches(dataset), desc=f"{flag} eval"):
            b_sentences1, b_sentences2, b_labels = batch['sentence1'], batch['sentence2'], batch['is_duplicate']
            truth.extend(b_labels)
            logits = model.forward(b_sentences1, b_sentences2, task, device)
            logits = logits.detach().cpu().numpy().flatten()
            if task == 0:
                new_predictions = np.round(logits).flatten()
            else:
                new_predictions = np.argmax(logits, axis=1).flatten()
            predictions.extend(new_predictions)
        accuracy = (np.array(truth).flatten() == np.array(predictions).flatten()).mean()
    return accuracy

def train_multitask_model():
    '''
    use AdamW optimizer.
    binary cross-entropy loss for para, multi-class cross-entropy loss for sts, sum loss functions. 
    make sure to save model at end to a specific path.
    '''
    pass 

def test_multitask_model():
    '''
    given dataloader, multitask finetuned model, and device
    return the accuracy for para and for sts
    '''
    #shawty
    pass

In [126]:
def test_singletask_model(filepath1, filepath2):
    train_singletask_para_model(para_train, para_dev, filepath1)
    # train_singletask_sts_model(sts_train, sts_dev, filepath2)

    device = torch.device('cuda')

    para_model, _ = load_model(filepath1, device)
    # sts_model, _ = load_model(filepath2, device)

    para_acc = eval_singletask_model(para_model, device, para_test, 0, 'test')
    # sts_acc = test_singletask_model(sts_model, device, sts_test, 1, 'test')

    print(f'Final test accuracy. PARA: {para_acc}.')
    # print(f'Final test accuracy. PARA: {para_acc}, STS: {sts_acc}.')

In [128]:
test_singletask_model('./models/para_single', './models/sts_single')

train eval: 4it [00:02,  1.50it/s]
dev eval: 1it [00:00, 20.71it/s]


epoch number: 0, para train accuracy: 0.635, para dev accuracy: 0.7


train: 4it [00:02,  1.52it/s]
train eval: 4it [00:02,  1.52it/s]
dev eval: 1it [00:00, 20.77it/s]


epoch number: 1, para train accuracy: 0.639, para dev accuracy: 0.7
New best model. Saving.


NotADirectoryError: [Errno 20] Not a directory: './models/para_single/transformer'